In [12]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from gpflow.kernels import Matern32
from gpflow.models import GPR
from gpflow import set_trainable
from sklearn.preprocessing import StandardScaler
import vectorbt as vbt

# Configuration to prevent scientific notation and set pandas display options
pd.set_option('display.float_format', lambda x: '%.6f' % x)

def fetch_and_process_data(ticker, start_date="2024-01-01", end_date="2024-12-31", train_ratio=0.8):
    """Fetch and process data with special handling for low-price assets."""
    try:
        data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=False)
        if data.empty:
            raise ValueError(f"No valid data found for {ticker} between {start_date} and {end_date}")
        if "Adj Close" in data.columns:
            close = data["Adj Close"]
        elif "Close" in data.columns:
            close = data["Close"]
        else:
            raise ValueError(f"Neither 'Adj Close' nor 'Close' column found for {ticker}")
        
        close = close.squeeze()
        split_index = int(len(close) * train_ratio)
        mean_close = close.mean()
        if isinstance(mean_close, pd.Series):
            mean_close = float(mean_close.iloc[0])
        else:
            mean_close = float(mean_close)
        price_scale = 1000 if mean_close < 0.1 else 1
        scaled_close = close * price_scale
        if mean_close > 0.1:
            returns = scaled_close.pct_change().dropna()
        else:
            returns = np.log(scaled_close / scaled_close.shift(1)).dropna()
        
        train_returns = returns.iloc[:split_index]
        test_returns = returns.iloc[split_index:]
        
        scaler = StandardScaler()
        scaler.fit(train_returns.values.reshape(-1, 1))
        train_std = scaler.transform(train_returns.values.reshape(-1, 1))
        test_std = scaler.transform(test_returns.values.reshape(-1, 1))
        std_returns = np.concatenate([train_std, test_std]).flatten()
        
        return close.reindex(returns.index), pd.Series(std_returns, index=returns.index), split_index
    except Exception as e:
        raise RuntimeError(f"Error processing {ticker} from {start_date} to {end_date}: {str(e)}")

def generate_signals(returns, split_index, epochs=50):
    """Generate trading signals with proper train-test separation."""
    X = np.arange(len(returns), dtype=np.float64).reshape(-1, 1)
    y = returns.values.reshape(-1, 1)
    
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, _ = y[:split_index], y[split_index:]
    
    kernel = Matern32()
    gpr = GPR(data=(X_train, y_train), kernel=kernel)
    set_trainable(gpr.likelihood.variance, False)
    
    trend_train = gpr.predict_f(X_train)[0].numpy().flatten()
    trend_test = gpr.predict_f(X_test)[0].numpy().flatten()
    trend = np.concatenate([trend_train, trend_test])
    
    features = np.hstack([returns.values.reshape(-1, 1), trend.reshape(-1, 1)])
    train_features = features[:split_index]
    train_labels = y[:split_index].flatten()
    
    model = Sequential([
        Input(shape=(1, 2)),
        LSTM(64, return_sequences=True),
        Dropout(0.2),
        LSTM(32),
        Dense(1, activation="tanh")
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")
    model.fit(
        train_features.reshape((train_features.shape[0], 1, train_features.shape[1])),
        train_labels,
        epochs=epochs,
        batch_size=32,
        verbose=0
    )
    
    all_features = features.reshape((features.shape[0], 1, features.shape[1]))
    predicted_signals = model.predict(all_features).flatten()
    
    return pd.Series(np.clip(predicted_signals, -1, 1), index=returns.index)

def backtest_strategy(close, positions):
    """Backtest strategy using generated signals."""
    entries = positions.gt(0.2)
    exits = positions.lt(-0.2)
    
    portfolio = vbt.Portfolio.from_signals(
        close=close,
        entries=entries,
        exits=exits,
        size=positions.abs(),
        freq="1D"
    )
    return portfolio

def run_backtest_for_period(ticker, start_date, end_date, description=""):
    """Run backtest for a given time period, display portfolio stats, and plot results."""
    print(f"\n--- Backtest for {ticker} - {description} ---")
    print(f"Period: {start_date} to {end_date}")
    try:
        close, returns, split_index = fetch_and_process_data(ticker, start_date, end_date)
        positions = generate_signals(returns, split_index)
        portfolio = backtest_strategy(close, positions)
        
        # Print summary metrics
        stats = portfolio.stats()
        print("\nPortfolio Statistics:")
        print(stats)
        
        # Print key metrics
        total_return = float(portfolio.total_return())
        sharpe_ratio = float(portfolio.sharpe_ratio())
        max_drawdown = float(portfolio.max_drawdown())
        print(f"\nKey Metrics:\nTotal Return: {total_return:.2%}\nSharpe Ratio: {sharpe_ratio:.2f}\nMax Drawdown: {max_drawdown:.2%}")
        
        # Plot portfolio performance and equity curve
        fig = portfolio.plot(title=f'Portfolio Performance - {description}').fig
        fig.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error for period {description}: {str(e)}")

def main():
    # Define the top 10 cryptocurrencies (exclude stable coins)
    # Please note that depending on your data source, some tickers might not have all required data.
    cryptocurrencies = {
        "Bitcoin": "BTC-USD",
        "Ethereum": "ETH-USD",
        "BNB": "BNB-USD",
        "XRP": "XRP-USD",
        "Cardano": "ADA-USD",
        "Solana": "SOL-USD",
        "Polkadot": "DOT-USD",
        "Dogecoin": "DOGE-USD",
        "Polygon": "MATIC-USD",
        "Litecoin": "LTC-USD"
    }
    
    # Define the periods for the backtest
    periods = [
        {"description": "2019 (1 year)", "start_date": "2019-01-01", "end_date": "2019-12-31"},
        {"description": "2020 (1 year)", "start_date": "2020-01-01", "end_date": "2020-12-31"},
        {"description": "2021 (1 year)", "start_date": "2021-01-01", "end_date": "2021-12-31"},
        {"description": "2022 (1 year)", "start_date": "2022-01-01", "end_date": "2022-12-31"},
        {"description": "2023 (1 year)", "start_date": "2023-01-01", "end_date": "2023-12-31"},
        {"description": "2024 (1 year)", "start_date": "2024-01-01", "end_date": "2024-12-31"},
        {"description": "2025 (YTD)", "start_date": "2025-01-01", "end_date": "2025-02-20"},
        {"description": "2-Year Window (2019-2020)", "start_date": "2019-01-01", "end_date": "2020-12-31"},
        {"description": "3-Year Window (2019-2021)", "start_date": "2019-01-01", "end_date": "2021-12-31"},
        {"description": "5-Year Window (2019-2023)", "start_date": "2019-01-01", "end_date": "2023-12-31"},
    ]

    
    for crypto_name, ticker in cryptocurrencies.items():
        print(f"\n\n=== Backtesting for {crypto_name} ({ticker}) ===")
        for period in periods:
            run_backtest_for_period(
                ticker,
                period["start_date"],
                period["end_date"],
                f"{crypto_name} - {period['description']}"
            )

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed



=== Backtesting for Bitcoin (BTC-USD) ===

--- Backtest for BTC-USD - Bitcoin - 2019 (1 year) ---
Period: 2019-01-01 to 2019-12-31


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

Portfolio Statistics:
Start                               2019-01-02 00:00:00
End                                 2019-12-30 00:00:00
Period                                363 days 00:00:00
Start Value                                  100.000000
End Value                                    208.138380
Total Return [%]                             108.138380
Benchmark Return [%]                          84.941362
Max Gross Exposure [%]                       100.000000
Total Fees Paid                                0.000000
Max Drawdown [%]                              21.784298
Max Drawdown Duration                  89 days 00:00:00
Total Trades                                         64
Total Closed Trades                                  64
Total Open Trades                                     0
Open Trade PnL                                 0.000000
Win Rate [%]                                  35.937500
Best Trade [%]                           

[*********************100%***********************]  1 of 1 completed

Error for period Bitcoin - 2019 (1 year): 'FigureWidget' object has no attribute 'fig'

--- Backtest for BTC-USD - Bitcoin - 2020 (1 year) ---
Period: 2020-01-01 to 2020-12-31


...

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step

Portfolio Statistics:
Start                               2019-01-02 00:00:00
End                                 2020-12-30 00:00:00
Period                                729 days 00:00:00
Start Value                                  100.000000
End Value                                    131.543587
Total Return [%]                              31.543587
Benchmark Return [%]                         287.232497
Max Gross Exposure [%]                       100.000000
Total Fees Paid                                0.000000
Max Drawdown [%]                              23.967415
Max Drawdown Duration                 538 days 00:00:00
Total Trades                                        159
Total Closed Trades                                 159
Total Open Trades                                     0
Open Trade PnL                                 0.000000
Win Rate [%]                                  34.591195
Best Trade [%]                           

[*********************100%***********************]  1 of 1 completed


35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step

Portfolio Statistics:
Start                               2019-01-02 00:00:00
End                                 2021-12-30 00:00:00
Period                               1094 days 00:00:00
Start Value                                  100.000000
End Value                                    138.645143
Total Return [%]                              38.645143
Benchmark Return [%]                         343.522831
Max Gross Exposure [%]                       100.000000
Total Fees Paid                                0.000000
Max Drawdown [%]                              62.197472
Max Drawdown Duration                 538 days 00:00:00
Total Trades                                        220
Total Closed Trades                                 219
Total Open Trades                                     1
Open Trade PnL                                 0.000000
Win Rate [%]                                  37.899543
Best Trade [%]                            

[*********************100%***********************]  1 of 1 completed


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 427us/step

Portfolio Statistics:
Start                               2019-01-02 00:00:00
End                                 2023-12-30 00:00:00
Period                               1824 days 00:00:00
Start Value                                  100.000000
End Value                                    101.096058
Total Return [%]                               1.096058
Benchmark Return [%]                         119.278085
Max Gross Exposure [%]                       100.000000
Total Fees Paid                                0.000000
Max Drawdown [%]                              75.302167
Max Drawdown Duration                 969 days 00:00:00
Total Trades                                        360
Total Closed Trades                                 360
Total Open Trades                                     0
Open Trade PnL                                 0.000000
Win Rate [%]                                  38.055556
Best Trade [%]                          

In [20]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from gpflow.kernels import Matern32
from gpflow.models import GPR
from gpflow import set_trainable
from sklearn.preprocessing import StandardScaler
import vectorbt as vbt

# Configuration to prevent scientific notation and set pandas display options
pd.set_option('display.float_format', lambda x: '%.6f' % x)

def fetch_and_process_data(ticker, start_date="2024-01-01", end_date="2024-12-31", train_ratio=0.8):
    """Fetch and process data with special handling for low-price assets."""
    try:
        data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=False)
        if data.empty:
            raise ValueError(f"No valid data found for {ticker} between {start_date} and {end_date}")
        if "Adj Close" in data.columns:
            close = data["Adj Close"]
        elif "Close" in data.columns:
            close = data["Close"]
        else:
            raise ValueError(f"Neither 'Adj Close' nor 'Close' column found for {ticker}")
        
        close = close.squeeze()
        split_index = int(len(close) * train_ratio)
        mean_close = close.mean()
        if isinstance(mean_close, pd.Series):
            mean_close = float(mean_close.iloc[0])
        else:
            mean_close = float(mean_close)
        price_scale = 1000 if mean_close < 0.1 else 1
        scaled_close = close * price_scale
        if mean_close > 0.1:
            returns = scaled_close.pct_change().dropna()
        else:
            returns = np.log(scaled_close / scaled_close.shift(1)).dropna()
        
        train_returns = returns.iloc[:split_index]
        test_returns = returns.iloc[split_index:]
        
        scaler = StandardScaler()
        scaler.fit(train_returns.values.reshape(-1, 1))
        train_std = scaler.transform(train_returns.values.reshape(-1, 1))
        test_std = scaler.transform(test_returns.values.reshape(-1, 1))
        std_returns = np.concatenate([train_std, test_std]).flatten()
        
        return close.reindex(returns.index), pd.Series(std_returns, index=returns.index), split_index
    except Exception as e:
        raise RuntimeError(f"Error processing {ticker} from {start_date} to {end_date}: {str(e)}")

def generate_signals(returns, split_index, epochs=50):
    """Generate trading signals with proper train-test separation."""
    X = np.arange(len(returns), dtype=np.float64).reshape(-1, 1)
    y = returns.values.reshape(-1, 1)
    
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, _ = y[:split_index], y[split_index:]
    
    kernel = Matern32()
    gpr = GPR(data=(X_train, y_train), kernel=kernel)
    set_trainable(gpr.likelihood.variance, False)
    
    trend_train = gpr.predict_f(X_train)[0].numpy().flatten()
    trend_test = gpr.predict_f(X_test)[0].numpy().flatten()
    trend = np.concatenate([trend_train, trend_test])
    
    features = np.hstack([returns.values.reshape(-1, 1), trend.reshape(-1, 1)])
    train_features = features[:split_index]
    train_labels = y[:split_index].flatten()
    
    model = Sequential([
        Input(shape=(1, 2)),
        LSTM(64, return_sequences=True),
        Dropout(0.2),
        LSTM(32),
        Dense(1, activation="tanh")
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")
    model.fit(
        train_features.reshape((train_features.shape[0], 1, train_features.shape[1])),
        train_labels,
        epochs=epochs,
        batch_size=32,
        verbose=0
    )
    
    all_features = features.reshape((features.shape[0], 1, features.shape[1]))
    predicted_signals = model.predict(all_features).flatten()
    
    return pd.Series(np.clip(predicted_signals, -1, 1), index=returns.index)

def backtest_strategy(close, positions):
    """Backtest strategy using generated signals."""
    entries = positions.gt(0.2)
    exits = positions.lt(-0.2)
    
    portfolio = vbt.Portfolio.from_signals(
        close=close,
        entries=entries,
        exits=exits,
        size=positions.abs(),
        freq="1D"
    )
    return portfolio

def run_backtest_for_period(ticker, start_date, end_date, description=""):
    """Run backtest for a given time period, display portfolio stats, and plot results."""
    print(f"\n--- Backtest for {ticker} - {description} ---")
    print(f"Period: {start_date} to {end_date}")
    try:
        close, returns, split_index = fetch_and_process_data(ticker, start_date, end_date)
        positions = generate_signals(returns, split_index)
        portfolio = backtest_strategy(close, positions)
        
        # Print summary metrics
        stats = portfolio.stats()
        print("\nPortfolio Statistics:")
        print(stats)
        
        # Print key metrics
        total_return = float(portfolio.total_return())
        sharpe_ratio = float(portfolio.sharpe_ratio())
        max_drawdown = float(portfolio.max_drawdown())
        print(f"\nKey Metrics:\nTotal Return: {total_return:.2%}\nSharpe Ratio: {sharpe_ratio:.2f}\nMax Drawdown: {max_drawdown:.2%}")
        
        print(portfolio.orders)
        print(portfolio.trades)

        # Plot portfolio performance and equity curve
        fig = portfolio.plot(title=f'Portfolio Performance - {description}').fig
        fig.tight_layout()
        plt.show()
        
        fig_value = portfolio.plot_value(title=f'Portfolio Value Over Time - {ticker}')
        plt.show()   # Using plt.show() instead of fig_value.show() for matplotlib figures
        
    except Exception as e:
        print(f"Error for period {description}: {str(e)}")

def main():
    ticker = "XRP-USD"
    periods = [
        {"description": "2019 (1 year)", "start_date": "2019-01-01", "end_date": "2019-12-31"},
        {"description": "2020 (1 year)", "start_date": "2020-01-01", "end_date": "2020-12-31"},
        {"description": "2021 (1 year)", "start_date": "2021-01-01", "end_date": "2021-12-31"},
        {"description": "2022 (1 year)", "start_date": "2022-01-01", "end_date": "2022-12-31"},
        {"description": "2023 (1 year)", "start_date": "2023-01-01", "end_date": "2023-12-31"},
        {"description": "2025 (YTD)", "start_date": "2025-01-01", "end_date": "2025-02-20"},
        {"description": "2-Year Window (2019-2020)", "start_date": "2019-01-01", "end_date": "2020-12-31"},
        {"description": "3-Year Window (2019-2021)", "start_date": "2019-01-01", "end_date": "2021-12-31"},
        {"description": "5-Year Window (2019-2023)", "start_date": "2019-01-01", "end_date": "2023-12-31"},
    ]
    
    for period in periods:
        run_backtest_for_period(
            ticker,
            period["start_date"],
            period["end_date"],
            period["description"]
        )

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed


--- Backtest for XRP-USD - 2019 (1 year) ---
Period: 2019-01-01 to 2019-12-31


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


[*********************100%***********************]  1 of 1 completed


Portfolio Statistics:
Start                               2019-01-02 00:00:00
End                                 2019-12-30 00:00:00
Period                                363 days 00:00:00
Start Value                                  100.000000
End Value                                     99.929308
Total Return [%]                              -0.070692
Benchmark Return [%]                         -48.173852
Max Gross Exposure [%]                         0.437388
Total Fees Paid                                0.000000
Max Drawdown [%]                               0.120269
Max Drawdown Duration                 229 days 00:00:00
Total Trades                                         75
Total Closed Trades                                  75
Total Open Trades                                     0
Open Trade PnL                                 0.000000
Win Rate [%]                                  29.333333
Best Trade [%]                                28.952144
Worst Trade [%]          

...

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step

Portfolio Statistics:
Start                               2019-01-02 00:00:00
End                                 2020-12-30 00:00:00
Period                                729 days 00:00:00
Start Value                                  100.000000
End Value                                     99.990087
Total Return [%]                              -0.009913
Benchmark Return [%]                         -43.549116
Max Gross Exposure [%]                         0.691377
Total Fees Paid                                0.000000
Max Drawdown [%]                               0.137785
Max Drawdown Duration                 555 days 00:00:00
Total Trades                                        153
Total Closed Trades                                 153
Total Open Trades                                     0
Open Trade PnL                                 0.000000
Win Rate [%]                                  31.372549
Best Trade [%]                            

[*********************100%***********************]  1 of 1 completed


35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step

Portfolio Statistics:
Start                               2019-01-02 00:00:00
End                                 2021-12-30 00:00:00
Period                               1094 days 00:00:00
Start Value                                  100.000000
End Value                                    100.304007
Total Return [%]                               0.304007
Benchmark Return [%]                         123.761404
Max Gross Exposure [%]                         1.749942
Total Fees Paid                                0.000000
Max Drawdown [%]                               0.678133
Max Drawdown Duration                 688 days 00:00:00
Total Trades                                        203
Total Closed Trades                                 202
Total Open Trades                                     1
Open Trade PnL                                 0.000000
Win Rate [%]                                  34.158416
Best Trade [%]                            

[*********************100%***********************]  1 of 1 completed


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step

Portfolio Statistics:
Start                               2019-01-02 00:00:00
End                                 2023-12-30 00:00:00
Period                               1824 days 00:00:00
Start Value                                  100.000000
End Value                                    100.093171
Total Return [%]                               0.093171
Benchmark Return [%]                          65.717676
Max Gross Exposure [%]                         1.816823
Total Fees Paid                                0.000000
Max Drawdown [%]                               0.839724
Max Drawdown Duration                 990 days 00:00:00
Total Trades                                        332
Total Closed Trades                                 332
Total Open Trades                                     0
Open Trade PnL                                 0.000000
Win Rate [%]                                  31.024096
Best Trade [%]                          

In [24]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from gpflow.kernels import Matern32, Kernel
from gpflow.models import GPR
from gpflow import set_trainable
from sklearn.preprocessing import StandardScaler
import vectorbt as vbt
from datetime import datetime, timedelta
import ruptures as rpt
from scipy.signal import find_peaks

class ChangePointKernel(Kernel):
    def __init__(self, base_kernel, changepoints):
        super().__init__()
        self.base_kernel = base_kernel
        # changepoints: list of indices where a regime change is detected.
        self.changepoints = changepoints

    def get_region(self, X):
        # Assumes X is an [N, D] tensor where the first column represents time or index.
        # Convert changepoints to a tensor of same dtype as X[:, 0]
        cp = tf.constant(self.changepoints, dtype=X.dtype)
        # Determine region index for each element in X using tf.searchsorted.
        regions = tf.searchsorted(cp, X[:, 0], side='right')
        return regions

    def K(self, X, X2=None):
        if X2 is None:
            X2 = X
        # Get region assignments for each input
        regions_X = self.get_region(X)
        regions_X2 = self.get_region(X2)
        # Create an indicator matrix where entries are 1 if both points belong to the same region, else 0.
        regions_equal = tf.cast(tf.equal(tf.expand_dims(regions_X, 1), tf.expand_dims(regions_X2, 0)), X.dtype)
        # Compute the base kernel covariance matrix.
        base_cov = self.base_kernel.K(X, X2)
        return base_cov * regions_equal

    def K_diag(self, X):
        return self.base_kernel.K_diag(X)

def fetch_and_process_data(
    ticker="BTC-USD",
    start_date=(datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d'),
    end_date=datetime.now().strftime('%Y-%m-%d')
):
    # Download data
    data = yf.download(ticker, start=start_date, end=end_date)
    
    if data.empty:
        raise ValueError(f"No data found for {ticker} between {start_date} and {end_date}")
    
    # Use "Adj Close" if available, otherwise fallback to "Close"
    if "Adj Close" in data.columns:
        price_series = data["Adj Close"].copy()
    else:
        price_series = data["Close"].copy()
    
    # Ensure price_series is a Series
    if isinstance(price_series, pd.DataFrame):
        price_series = price_series.squeeze()
    
    price_series.name = ticker
    returns = price_series.pct_change().dropna()
    
    scaler = StandardScaler()
    std_returns = scaler.fit_transform(returns.values.reshape(-1, 1))
    
    valid_index = returns.index
    return price_series.loc[valid_index], pd.Series(
        std_returns.flatten(), index=valid_index, name="Standardized Returns"
    )

def detect_changepoints(returns):
    # Use ruptures to detect changepoints in the return series.
    algo = rpt.Pelt(model="rbf").fit(returns.values)
    result = algo.predict(pen=10)
    return result

def generate_signals(returns, changepoints, train_ratio=0.8, epochs=50, lookback_window=30):
    # Prepare features: using time step as a feature
    X = np.arange(len(returns), dtype=np.float64).reshape(-1, 1)
    y = returns.values.reshape(-1, 1)
    
    # Use GPR with ChangePointKernel for trend detection
    base_kernel = Matern32()
    kernel = ChangePointKernel(base_kernel, changepoints)
    gpr = GPR(data=(X, y), kernel=kernel)
    set_trainable(gpr.likelihood.variance, False)
    trend = gpr.predict_f(X)[0].numpy().flatten()
    
    # Combine returns and trend to form features
    features = np.hstack([returns.values.reshape(-1, 1), trend.reshape(-1, 1)])
    
    # Optionally, use lookback_window to create a dynamic feature window if desired.
    # (For example, calculate rolling averages to smooth the features.)
    
    # Train/test split (only use train_features for training)
    split_index = int(len(features) * train_ratio)
    train_features = features[:split_index]
    train_labels = returns.values[:split_index]
    
    # Reshape for LSTM input (3D shape required)
    train_features = train_features.reshape(
        (train_features.shape[0], 1, train_features.shape[1])
    )
    
    # Build LSTM model
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(train_features.shape[1], train_features.shape[2])),
        Dropout(0.2),
        LSTM(32),
        Dense(1, activation="tanh")
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")
    
    # Fit the model on the training portion
    model.fit(train_features, train_labels, epochs=epochs, batch_size=32, verbose=0)
    
    # Predict signals using all features
    all_features = features.reshape((features.shape[0], 1, features.shape[1]))
    predicted_signals = model.predict(all_features, verbose=0).flatten()
    
    # Clip predictions to the range [-1, 1]
    positions = np.clip(predicted_signals, -1, 1)
    
    return positions

def backtest_strategy(price_series, positions):
    # Ensure price_series is a Series with a name
    if isinstance(price_series, pd.DataFrame):
        price_series = price_series.squeeze()
    
    positions = pd.Series(positions, index=price_series.index, name=price_series.name)
    entries = positions > 0
    exits = positions < 0
    
    # Create the portfolio using vectorbt
    pf = vbt.Portfolio.from_signals(
        close=price_series, 
        entries=entries, 
        exits=exits, 
        size=np.abs(positions), 
        freq="1D"
    )
    return pf

def evaluate_performance(portfolio):
    # Print portfolio metrics and plot performance
    print("\nPortfolio Metrics:")
    print(f"Total Return: {float(portfolio.total_return()):.2%}")
    print(f"Sharpe Ratio: {float(portfolio.sharpe_ratio()):.2f}")
    print(f"Max Drawdown: {float(portfolio.max_drawdown()):.2%}")
    
    # Plot portfolio value
    fig_value = portfolio.plot_value(
        title=f'Portfolio Value Over Time - {portfolio.close.name}'
    )
    fig_value.show()
    
    # Plot drawdowns
    fig_drawdowns = portfolio.plot_drawdowns(
        title=f'Portfolio Drawdowns - {portfolio.close.name}'
    )
    fig_drawdowns.show()

def main():
    ticker = "BTC-USD"
    
    # Fetch and process data
    price_series, returns = fetch_and_process_data(
        ticker=ticker, 
        start_date="2022-01-01", 
        end_date="2023-01-01"
    )
    
    # Detect changepoints
    changepoints = detect_changepoints(returns)
    
    # Generate trading signals
    positions = generate_signals(returns, changepoints, train_ratio=0.8, epochs=50, lookback_window=30)
    
    # Backtest strategy
    portfolio = backtest_strategy(price_series, positions)
    
    # Evaluate performance
    evaluate_performance(portfolio)
    
    print(portfolio.stats())
 
if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.




Portfolio Metrics:
Total Return: -26.40%
Sharpe Ratio: -0.55
Max Drawdown: -41.40%


Start                               2022-01-02 00:00:00
End                                 2022-12-31 00:00:00
Period                                364 days 00:00:00
Start Value                                  100.000000
End Value                                     73.601326
Total Return [%]                             -26.398674
Benchmark Return [%]                         -65.049277
Max Gross Exposure [%]                       100.000000
Total Fees Paid                                0.000000
Max Drawdown [%]                              41.397141
Max Drawdown Duration                 297 days 00:00:00
Total Trades                                         88
Total Closed Trades                                  88
Total Open Trades                                     0
Open Trade PnL                                 0.000000
Win Rate [%]                                  36.363636
Best Trade [%]                                17.253614
Worst Trade [%]                               -9

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from gpflow.kernels import Matern32, Kernel
from gpflow.models import GPR
from gpflow import set_trainable
from sklearn.preprocessing import StandardScaler
import vectorbt as vbt
from datetime import datetime, timedelta
import ruptures as rpt
from scipy.signal import find_peaks
from sklearn.model_selection import ParameterGrid

class ChangePointKernel(Kernel):
    def __init__(self, base_kernel, changepoints):
        super().__init__()
        self.base_kernel = base_kernel
        self.changepoints = changepoints

    def get_region(self, X):
        cp = tf.constant(self.changepoints, dtype=X.dtype)
        regions = tf.searchsorted(cp, X[:, 0], side='right')
        return regions

    def K(self, X, X2=None):
        if X2 is None:
            X2 = X
        regions_X = self.get_region(X)
        regions_X2 = self.get_region(X2)
        regions_equal = tf.cast(tf.equal(tf.expand_dims(regions_X, 1), tf.expand_dims(regions_X2, 0)), X.dtype)
        base_cov = self.base_kernel.K(X, X2)
        return base_cov * regions_equal

    def K_diag(self, X):
        return self.base_kernel.K_diag(X)

def fetch_and_process_data(
    ticker="BTC-USD",
    start_date=(datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d'),
    end_date=datetime.now().strftime('%Y-%m-%d')
):
    data = yf.download(ticker, start=start_date, end=end_date)
    if data.empty:
        raise ValueError(f"No data found for {ticker} between {start_date} and {end_date}")
    if "Adj Close" in data.columns:
        price_series = data["Adj Close"].copy()
    else:
        price_series = data["Close"].copy()
    if isinstance(price_series, pd.DataFrame):
        price_series = price_series.squeeze()
    price_series.name = ticker
    returns = price_series.pct_change().dropna()
    scaler = StandardScaler()
    std_returns = scaler.fit_transform(returns.values.reshape(-1, 1))
    valid_index = returns.index
    return price_series.loc[valid_index], pd.Series(
        std_returns.flatten(), index=valid_index, name="Standardized Returns"
    )

def detect_changepoints(returns, lookback_window=21):
    algo = rpt.Pelt(model="rbf").fit(returns.values)
    result = algo.predict(pen=lookback_window)
    return result

def generate_signals(returns, changepoints, train_ratio=0.8, epochs=50, lookback_window=30, dropout_rate=0.2, lstm_hidden_units=64, learning_rate=0.001, batch_size=32):
    X = np.arange(len(returns), dtype=np.float64).reshape(-1, 1)
    y = returns.values.reshape(-1, 1)
    base_kernel = Matern32()
    kernel = ChangePointKernel(base_kernel, changepoints)
    gpr = GPR(data=(X, y), kernel=kernel)
    set_trainable(gpr.likelihood.variance, False)
    trend = gpr.predict_f(X)[0].numpy().flatten()
    features = np.hstack([returns.values.reshape(-1, 1), trend.reshape(-1, 1)])
    split_index = int(len(features) * train_ratio)
    train_features = features[:split_index]
    train_labels = returns.values[:split_index]
    train_features = train_features.reshape((train_features.shape[0], 1, train_features.shape[1]))
    model = Sequential([
        LSTM(lstm_hidden_units, return_sequences=True, input_shape=(train_features.shape[1], train_features.shape[2])),
        Dropout(dropout_rate),
        LSTM(int(lstm_hidden_units/2)),
        Dense(1, activation="tanh")
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
    model.fit(train_features, train_labels, epochs=epochs, batch_size=batch_size, verbose=0)
    all_features = features.reshape((features.shape[0], 1, features.shape[1]))
    predicted_signals = model.predict(all_features, verbose=0).flatten()
    positions = np.clip(predicted_signals, -1, 1)
    return positions

def backtest_strategy(price_series, positions):
    if isinstance(price_series, pd.DataFrame):
        price_series = price_series.squeeze()
    positions = pd.Series(positions, index=price_series.index, name=price_series.name)
    entries = positions > 0
    exits = positions < 0
    pf = vbt.Portfolio.from_signals(
        close=price_series, 
        entries=entries, 
        exits=exits, 
        size=np.abs(positions), 
        freq="1D"
    )
    return pf

def evaluate_performance(portfolio):
    print("\nPortfolio Metrics:")
    print(f"Total Return: {float(portfolio.total_return()):.2%}")
    print(f"Sharpe Ratio: {float(portfolio.sharpe_ratio()):.2f}")
    print(f"Max Drawdown: {float(portfolio.max_drawdown()):.2%}")
    fig_value = portfolio.plot_value(
        title=f'Portfolio Value Over Time - {portfolio.close.name}'
    )
    fig_value.show()
    fig_drawdowns = portfolio.plot_drawdowns(
        title=f'Portfolio Drawdowns - {portfolio.close.name}'
    )
    fig_drawdowns.show()

def main():
    ticker = "BTC-USD"
    price_series, returns = fetch_and_process_data(
        ticker=ticker, 
        start_date="2024-01-01", 
        end_date="2025-01-01"
    )
    LBW_range = [10, 21, 63, 126, 252]
    dropout_rate_range = [0.1, 0.2, 0.3, 0.4, 0.5]
    hidden_units_range = [5, 10, 20, 40, 80, 160]
    batch_size_range = [64, 128, 256]
    learning_rate_range = [1e-4, 1e-3, 1e-2, 1e-1]
    param_grid = {
        'lookback_window': LBW_range,
        'dropout_rate': dropout_rate_range,
        'lstm_hidden_units': hidden_units_range,
        'batch_size': batch_size_range,
        'learning_rate': learning_rate_range
    }
    best_sharpe_ratio = -np.inf
    best_params = None
    best_portfolio = None
    for params in ParameterGrid(param_grid):
        changepoints = detect_changepoints(returns, lookback_window=params['lookback_window'])
        positions = generate_signals(
            returns, changepoints,
            train_ratio=0.8, 
            epochs=50, 
            lookback_window=params['lookback_window'],
            dropout_rate=params['dropout_rate'],
            lstm_hidden_units=params['lstm_hidden_units'],
            learning_rate=params['learning_rate'],
            batch_size=params['batch_size']
        )
        portfolio = backtest_strategy(price_series, positions)
        sharpe_ratio = float(portfolio.sharpe_ratio())
        if sharpe_ratio > best_sharpe_ratio:
            best_sharpe_ratio = sharpe_ratio
            best_params = params
            best_portfolio = portfolio
    print(f"Best Parameters: {best_params}")
    evaluate_performance(best_portfolio)
    print(best_portfolio.stats())

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



KeyboardInterrupt: 

: 